# Expenses in closed companies
Recently we found out that there are many companies that are already closed or out of service, we are aiming to find if there are expenses made after the company situation as other than open.

In [1]:
import pandas as pd
import numpy as np
from serenata_toolbox.datasets import fetch

In [ ]:
fetch('2016-09-03-companies.xz', '../data')
fetch('2016-11-19-reimbursements.xz', '../data')

In [2]:
companies = pd.read_csv('../data/2016-09-03-companies.xz', low_memory=False)
reimbursements = pd.read_csv('../data/2016-11-19-reimbursements.xz',
                      dtype={'applicant_id': np.str,
                             'cnpj_cpf': np.str,
                             'congressperson_id': np.str,
                             'subquota_number': np.str},
                      low_memory=False)

In [3]:
companies.head(3)

,situation_date,type,name,phone,situation,neighborhood,address,number,zip_code,city,...,secondary_activity_97_code,secondary_activity_98,secondary_activity_98_code,secondary_activity_99,secondary_activity_99_code,secondary_activity_9_code,latitude,longitude,latitude.1,longitude.1
0,03/11/2005,MATRIZ,COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA CAER,(95) 3626-5165,ATIVA,SAO PEDRO,R MELVIN JONES,219,69.306-610,BOA VISTA,...,NaN,NaN,NaN,NaN,NaN,NaN,2.827880,-60.660094,2.827880,-60.660094
1,25/07/2012,MATRIZ,MG TERMINAIS RODOVIARIOS LTDA.,(54) 3218-3032,ATIVA,CENTRO,R ERNESTO ALVES,1341,95.020-360,CAXIAS DO SUL,...,NaN,NaN,NaN,NaN,NaN,NaN,-29.163731,-51.175180,-29.163731,-51.175180
2,09/11/2004,MATRIZ,POSTO ROTA 116 DERIVADOS DE PETROLEO LTDA,(51) 3561-2029 / (51) 3561-2029,ATIVA,PRIMAVERA,R LATERAL A RODOVIA BR 116,S/N,93.950-000,DOIS IRMAOS,...,NaN,NaN,NaN,NaN,NaN,NaN,-29.585206,-51.093225,-29.585206,-51.093225


In [4]:
reimbursements.head(3)

,year,applicant_id,document_id,reimbursement_value_total,total_net_value,reimbursement_numbers,congressperson_name,congressperson_id,congressperson_document,term,...,issue_date,document_value,remark_value,net_values,month,installment,passenger,leg_of_the_trip,batch_number,reimbursement_values
0,2009,1001,1564212,NaN,130.0,2888,DILCEU SPERAFICO,73768,444.0,2015.0,...,2009-04-06T00:00:00,130.00,0.00,130.0,4,0,NaN,NaN,388810,NaN
1,2009,1001,1564223,NaN,135.0,2888,DILCEU SPERAFICO,73768,444.0,2015.0,...,2009-04-07T00:00:00,141.75,6.75,135.0,4,0,NaN,NaN,388810,NaN
2,2009,1001,1568039,NaN,1000.0,2893,DILCEU SPERAFICO,73768,444.0,2015.0,...,2009-03-12T00:00:00,1000.00,0.00,1000.0,3,0,NaN,NaN,390159,NaN


## Formatting
Formatting companies situation_date and reimbursements issue_date columns to correct date format (will be needed for a query later), and formatting the companies cpnj to a format without dash and dots.

In [5]:
df = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                   'day': [4, 5],})
pd.to_datetime(df)

0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]

In [6]:
reimbursements.iloc[0]['issue_date']

'2009-04-06T00:00:00'

In [7]:
reimbursements['issue_date'] = pd.to_datetime(reimbursements['issue_date'],
                                              errors='coerce')
companies['situation_date'] = pd.to_datetime(companies['situation_date'],
                                             errors='coerce')

In [8]:
reimbursements.iloc[0]['issue_date']

Timestamp('2009-04-06 00:00:00')

In [9]:
companies.iloc[0]['cnpj']

'05.939.467/0001-15'

In [10]:
companies['cnpj'] = companies['cnpj'].str.replace(r'\D', '')
companies.iloc[0]['cnpj']

'05939467000115'

## Not 'ABERTA'

In [11]:
statuses = ['BAIXADA', 'NULA', 'SUSPENSA', 'INAPTA']
not_open = companies[companies['situation'].isin(statuses)]
not_open[['cnpj', 'situation_date','situation']].head(5)

,cnpj,situation_date,situation
37,03956142000115,2005-09-20,BAIXADA
248,08594693000108,2016-06-28,BAIXADA
329,20768047000107,2016-12-04,BAIXADA
364,03380051000346,2016-05-01,BAIXADA
395,17479634000171,2016-06-28,BAIXADA


In [12]:
not_open.shape

(2211, 228)

The column situation_date is the one that is interesting. Expenses made after that date should be considered suspicious.

The inner join on merge will give reimbursements that were requested for out of service companies.

In [13]:
dataset = pd.merge(reimbursements,
                   not_open,
                   left_on='cnpj_cpf',
                   right_on='cnpj')

In [14]:
dataset.shape

(93133, 259)

In [15]:
columns = ['congressperson_name',
           'issue_date','cnpj',
           'situation_date',
           'situation']
dataset[columns].head(10)

,congressperson_name,issue_date,cnpj,situation_date,situation
0,DILCEU SPERAFICO,2009-04-06,02989654001197,2013-01-03,BAIXADA
1,DILCEU SPERAFICO,2009-09-23,02989654001197,2013-01-03,BAIXADA
2,DOMINGOS DUTRA,2009-10-14,02989654001197,2013-01-03,BAIXADA
3,EDINHO BEZ,2009-10-19,02989654001197,2013-01-03,BAIXADA
4,HERMES PARCIANELLO,2009-05-29,02989654001197,2013-01-03,BAIXADA
5,JAIME MARTINS,2009-04-08,02989654001197,2013-01-03,BAIXADA
6,JOSÉ CARLOS VIEIRA,2009-07-01,02989654001197,2013-01-03,BAIXADA
7,PAULO BORNHAUSEN,2009-03-26,02989654001197,2013-01-03,BAIXADA
8,PAULO BORNHAUSEN,2009-04-07,02989654001197,2013-01-03,BAIXADA
9,PAULO BORNHAUSEN,2009-04-28,02989654001197,2013-01-03,BAIXADA


In [16]:
dataset.iloc[0]

year                                                       2009
applicant_id                                               1001
document_id                                             1564212
reimbursement_value_total                                   NaN
total_net_value                                             130
reimbursement_numbers                                      2888
congressperson_name                            DILCEU SPERAFICO
congressperson_id                                         73768
congressperson_document                                     444
term                                                       2015
state_x                                                      PR
party                                                        PP
term_id                                                      55
subquota_number                                               3
subquota_description                       Fuels and lubricants
subquota_group_id                       

## Filtering suspicious reimbursements
We have all reibursements requested for expenses made in companies that have situation other than "open".
It is still necessary to check the reimbursement issue_date is "bigger" than the situation_date.

In [17]:
expenses_in_closed_companies = dataset.query('issue_date > situation_date')
expenses_in_closed_companies[columns].head()

,congressperson_name,issue_date,cnpj,situation_date,situation
2429,EDINHO ARAÚJO,2013-01-30,02989654001197,2013-01-03,BAIXADA
2430,EDINHO ARAÚJO,2013-02-02,02989654001197,2013-01-03,BAIXADA
2431,EDINHO ARAÚJO,2013-02-26,02989654001197,2013-01-03,BAIXADA
2432,EDINHO ARAÚJO,2013-03-01,02989654001197,2013-01-03,BAIXADA
2433,HERMES PARCIANELLO,2013-01-28,02989654001197,2013-01-03,BAIXADA


In [18]:
expenses_in_closed_companies.shape

(5222, 259)

We can safely say that there are 5222 suspicious reimbursements.